In [36]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import random
import pyautogui
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [37]:
#Functions

def configurar_driver():
    chrome_options = webdriver.ChromeOptions()
    options = chrome_options  # Use 'options' instead of 'chrome_options' for newer versions
    return webdriver.Chrome(options=options)

def abrir_pagina(driver, video_url):
    driver.get(video_url)
    driver.maximize_window()
    time.sleep(15)

def obtener_numero_comentarios(driver):
    return driver.find_elements(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')

def hacer_scroll_cargar_comentarios(driver, initial_comments):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')))
    time.sleep(2)
    current_comments = driver.find_elements(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')
    if current_comments == initial_comments:
        return True
    return False

def obtener_info_comentario(comentario_container):
    nombre_usuario_tag = comentario_container.find('a', href=True)
    if nombre_usuario_tag:
        nombre_usuario_href = nombre_usuario_tag.get('href')
        nombre_usuario = nombre_usuario_href.split('/')[-1]
    else:
        nombre_usuario = "No se encontró el nombre de usuario"

    fecha_tag = comentario_container.find('span', class_='css-1esugaz-SpanCreatedTime e1g2efjf8')
    fecha = fecha_tag.text if fecha_tag else "No se encontró la fecha"

    like_tag = comentario_container.find('span', class_='css-gb2mrc-SpanCount ezxoskx3')
    likes = like_tag.text if like_tag else "No se encontró la cantidad de likes"

    comentario_tag = comentario_container.find('p', class_='css-xm2h10-PCommentText e1g2efjf6')
    comentario_texto = comentario_tag.text if comentario_tag else "No se encontró el comentario"

    return nombre_usuario, fecha, likes, comentario_texto

def escribir_csv(comentario_containers):
    with open('comentarios_tiktok.csv', mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nombre de usuario', 'Fecha del comentario', 'Cantidad de likes', 'Comentario'])

        for comentario_container in comentario_containers:
            nombre_usuario, fecha, likes, comentario_texto = obtener_info_comentario(comentario_container)
            writer.writerow([nombre_usuario, fecha, likes, comentario_texto])
            print(f"Nombre de usuario: {nombre_usuario}")
            print(f"Fecha del comentario: {fecha}")
            print(f"Cantidad de likes: {likes}")
            print(f"Comentario: {comentario_texto}")
            print("------")

            respuesta_containers = comentario_container.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')
            for respuesta_container in respuesta_containers:
                nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp = obtener_info_comentario(respuesta_container)
                writer.writerow([nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp])
                print(f"Respuesta - Nombre de usuario: {nombre_usuario_resp}")
                print(f"Respuesta - Fecha del comentario: {fecha_resp}")
                print(f"Respuesta - Cantidad de likes: {likes_resp}")
                print(f"Respuesta - Comentario: {comentario_texto_resp}")
                print("------")

def escribir_csv2(comentario_containers):
    with open('comentarios_tiktok.csv', mode='a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)

        for comentario_container in comentario_containers:
            nombre_usuario, fecha, likes, comentario_texto = obtener_info_comentario(comentario_container)
            writer.writerow([nombre_usuario, fecha, likes, comentario_texto])
            print(f"Nombre de usuario: {nombre_usuario}")
            print(f"Fecha del comentario: {fecha}")
            print(f"Cantidad de likes: {likes}")
            print(f"Comentario: {comentario_texto}")
            print("------")

            respuesta_containers = comentario_container.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')
            for respuesta_container in respuesta_containers:
                nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp = obtener_info_comentario(respuesta_container)
                writer.writerow([nombre_usuario_resp, fecha_resp, likes_resp, comentario_texto_resp])
                print(f"Respuesta - Nombre de usuario: {nombre_usuario_resp}")
                print(f"Respuesta - Fecha del comentario: {fecha_resp}")
                print(f"Respuesta - Cantidad de likes: {likes_resp}")
                print(f"Respuesta - Comentario: {comentario_texto_resp}")
                print("------")

def escribir_csv3(comentario_containers):
    with open('comentarios_tiktok.csv', mode='a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        
        # Crear un conjunto para almacenar los comentarios existentes en el archivo CSV
        existing_comments = set()
        with open('comentarios_tiktok.csv', mode='r', encoding='utf-8', newline='') as read_file:
            reader = csv.reader(read_file)
            next(reader)  # Saltar la primera fila (encabezados)
            for row in reader:
                existing_comments.add(tuple(row))

        for comentario_container in comentario_containers:
            nombre_usuario, fecha, likes, comentario_texto = obtener_info_comentario(comentario_container)
            comment_tuple = (nombre_usuario, fecha, likes, comentario_texto)
            # Verificar si el comentario ya existe en el conjunto de comentarios existentes
            if comment_tuple not in existing_comments:
                writer.writerow([nombre_usuario, fecha, likes, comentario_texto])
                print(f"Nombre de usuario: {nombre_usuario}")
                print(f"Fecha del comentario: {fecha}")
                print(f"Cantidad de likes: {likes}")
                print(f"Comentario: {comentario_texto}")
                print("------")
                existing_comments.add(comment_tuple)

def main(iteraciones = 2):
    video_url = "https://www.tiktok.com/@rosieeeeriderrrrr/video/7327037613456575775"

    driver = configurar_driver()
    abrir_pagina(driver, video_url)

    initial_comments = obtener_numero_comentarios(driver)

    while True:
        if hacer_scroll_cargar_comentarios(driver, initial_comments):
            break

    html_code = driver.page_source
    soup = BeautifulSoup(html_code, 'html.parser')
    comentario_containers = soup.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')

    escribir_csv2(comentario_containers)

    print("La información se ha guardado en 'comentarios_tiktok.csv'")

    # Simular presionar la tecla de flecha hacia abajo para cambiar al siguiente video
    actions = webdriver.ActionChains(driver)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(5)
    #driver.quit()

def main2(iteraciones=1):
    driver = configurar_driver()  # Inicializar el controlador una vez fuera del bucle
    
    video_url = "https://www.tiktok.com/@gran_hermano_videos/video/7334923258061475077"  # URL del primer video
    abrir_pagina(driver, video_url)
    
    for _ in range(iteraciones):
        initial_comments = obtener_numero_comentarios(driver)
        
        while True:
            if hacer_scroll_cargar_comentarios2(driver, initial_comments):
               break
        
        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')
        comentario_containers = soup.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')
        
        escribir_csv2(comentario_containers)
        
        print("La información se ha guardado en 'comentarios_tiktok.csv'")
        
        # Simular desplazamiento hacia abajo para cargar el siguiente video
        actions = webdriver.ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(5)
        
        # Actualizar el número inicial de comentarios para el nuevo video
        initial_comments = obtener_numero_comentarios(driver)
    
    driver.quit()  # Cerrar el controlador una vez fuera del bucle

def hacer_scroll_cargar_comentarios2(driver, initial_comments):
    """
    Esta función carga comentarios en una página web evitando la carga infinita de videos.

    Args:
        driver (webdriver): El controlador del navegador web.
        initial_comments (list): Una lista de elementos de comentarios iniciales (opcional).

    Returns:
        bool: True si se han cargado nuevos comentarios, False en caso contrario.
    """

    previous_comment_count = 0  # Número de comentarios encontrados previamente
    max_iterations = 10  # Máximo número de iteraciones para evitar bucles infinitos

    for _ in range(max_iterations):
        # Encontrar el primer elemento de comentario para rastrear cambios
        primer_comentario = driver.find_element(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')

        # Desplazarse hacia abajo y esperar nuevos comentarios
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer')))
        time.sleep(2)

        # Contar la cantidad actual de comentarios
        cantidad_comentarios_actual = len(driver.find_elements(By.CSS_SELECTOR, 'div.css-1i7ohvi-DivCommentItemContainer'))

        # Verificar si se han cargado nuevos comentarios
        if cantidad_comentarios_actual == previous_comment_count:
            break  # Salir del bucle si no hay nuevos comentarios

        previous_comment_count = cantidad_comentarios_actual

    # Extraer comentarios usando información obtenida (primer_comentario, cantidad_comentarios_actual)
    # ... (código para extraer comentarios)

    return True  # Se han cargado nuevos comentarios

def type_with_random_delay(text):
    for char in text:
        pyautogui.typewrite(char)
        time.sleep(random.uniform(0.1, 0.3))  # Ajusta los valores según sea necesario

def performActions(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)
    
    if input_string:
        type_with_random_delay(input_string)
        time.sleep(random.random())
        pyautogui.press("enter")

def move_mouse_and_click(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    pyautogui.press("enter")
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)

def move_mouse_random_straight_line(target_x, target_y):
    current_x, current_y = pyautogui.position()

    deviation_range = 2

    # Calculate the distance between current and target position
    distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

    # Number of steps to take in the straight line trajectory
    num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
    if num_steps == 0:
        num_steps = 1  # Ajusta el tamaño del paso según sea necesario

    # Calculate step size in x and y directions
    step_x = (target_x - current_x) / num_steps
    step_y = (target_y - current_y) / num_steps

    # To stop the process when is close enough
    stop_process = 0

    # Move the mouse in a straight line trajectory
    while current_x != target_x or current_y != target_y:
        current_x, current_y = pyautogui.position()

        # Calculate the distance between current and target position
        distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

        # Number of steps to take in the straight line trajectory
        num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
        if num_steps == 0:
            # To stop the process when is close enough
            stop_process = stop_process + 1
            num_steps = 1  # Ajusta el tamaño del paso según sea necesario
            deviation_range = 1

        # Calculate step size in x and y directions
        step_x = (target_x - current_x) / num_steps
        step_y = (target_y - current_y) / num_steps

        current_x += step_x + random.randint(-deviation_range, deviation_range)
        current_y += step_y + random.randint(-deviation_range, deviation_range)
        pyautogui.moveTo(current_x, current_y, duration=0.01)

In [ ]:
main2(2)

In [ ]:
'''
def main(urls, iteraciones=2):
    for video_url in urls:
        driver = configurar_driver()
        abrir_pagina(driver, video_url)

        initial_comments = obtener_numero_comentarios(driver)

        while True:
            if hacer_scroll_cargar_comentarios2(driver, initial_comments):
                break

        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')
        comentario_containers = soup.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')

        escribir_csv2(comentario_containers)

        print(f"La información del video {video_url} se ha guardado en 'comentarios_tiktok.csv'")

        # Simular presionar la tecla de flecha hacia abajo para cambiar al siguiente video
        #actions = webdriver.ActionChains(driver)
        #actions.send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(5)

    driver.quit()

# Ejemplo de uso
urls = [
    "https://www.tiktok.com/@rosieeeeriderrrrr/video/7327037613456575775",
    "https://www.tiktok.com/@asesor.legal.en/video/7265162676047842566?q=milei%20sin%20peluca&t=1707928089921"
]
main(urls)
'''

In [53]:
def configurar_driver():
    chrome_options = webdriver.ChromeOptions()
    options = chrome_options
    return webdriver.Chrome(options=options)

def abrir_pagina(driver, url):
    driver.get(url)
    driver.maximize_window()
    time.sleep(15)

def extraer_enlaces_videos(driver, cantidad):
    enlaces_videos = []
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    div_videos = soup.find_all('div', class_='css-1soki6-DivItemContainerForSearch')
    for div_video in div_videos[:cantidad]:  # Limitar la cantidad de enlaces a extraer
        enlace = div_video.find('a')['href']
        enlaces_videos.append(enlace)
    return enlaces_videos

def extraccion_link(palabra, cantidad):
    driver = configurar_driver()  # Inicializar el controlador una vez fuera del bucle
    video_url = "https://www.tiktok.com/"  # URL del primer video
    abrir_pagina(driver, video_url)
    time.sleep(5)
    performActions(698, 188, '#milei')
    cantidad = 10
    time.sleep(5)
    enlaces = extraer_enlaces_videos(driver, cantidad)
    driver.quit()
    return enlaces

def type_with_random_delay(text):
    for char in text:
        pyautogui.typewrite(char)
        time.sleep(random.uniform(0.1, 0.3))  # Ajusta los valores según sea necesario

def performActions(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)
    
    if input_string:
        type_with_random_delay(input_string)
        time.sleep(random.random())
        pyautogui.press("enter")

def move_mouse_and_click(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    pyautogui.press("enter")
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)

def move_mouse_random_straight_line(target_x, target_y):
    current_x, current_y = pyautogui.position()

    deviation_range = 2

    # Calculate the distance between current and target position
    distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

    # Number of steps to take in the straight line trajectory
    num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
    if num_steps == 0:
        num_steps = 1  # Ajusta el tamaño del paso según sea necesario

    # Calculate step size in x and y directions
    step_x = (target_x - current_x) / num_steps
    step_y = (target_y - current_y) / num_steps

    # To stop the process when is close enough
    stop_process = 0

    # Move the mouse in a straight line trajectory
    while current_x != target_x or current_y != target_y:
        current_x, current_y = pyautogui.position()

        # Calculate the distance between current and target position
        distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

        # Number of steps to take in the straight line trajectory
        num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
        if num_steps == 0:
            # To stop the process when is close enough
            stop_process = stop_process + 1
            num_steps = 1  # Ajusta el tamaño del paso según sea necesario
            deviation_range = 1

        # Calculate step size in x and y directions
        step_x = (target_x - current_x) / num_steps
        step_y = (target_y - current_y) / num_steps

        current_x += step_x + random.randint(-deviation_range, deviation_range)
        current_y += step_y + random.randint(-deviation_range, deviation_range)
        pyautogui.moveTo(current_x, current_y, duration=0.01)

def main(enlaces, iteraciones=2):
    for video_url in enlaces:
        driver = configurar_driver()
        abrir_pagina(driver, video_url)

        initial_comments = obtener_numero_comentarios(driver)

        while True:
            if hacer_scroll_cargar_comentarios2(driver, initial_comments):
                break

        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')
        comentario_containers = soup.find_all('div', class_='css-1i7ohvi-DivCommentItemContainer')

        escribir_csv2(comentario_containers)

        print(f"La información del video {video_url} se ha guardado en 'comentarios_tiktok.csv'")

        # Simular presionar la tecla de flecha hacia abajo para cambiar al siguiente video
        #actions = webdriver.ActionChains(driver)
        #actions.send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(5)

    driver.quit()


['https://www.tiktok.com/@leonidaslibertario/video/7335508511683562757', 'https://www.tiktok.com/@elpelucamilei24/video/7335512019715509509', 'https://www.tiktok.com/@elpelucamilei/video/7332571358687137030', 'https://www.tiktok.com/@actualidadrt/video/7323275393639746822', 'https://www.tiktok.com/@leonidaslibertario/video/7335506318112886021', 'https://www.tiktok.com/@leonidaslibertario/video/7335519207381044486', 'https://www.tiktok.com/@patrioti_giorgiameloni/video/7334859142902877473', 'https://www.tiktok.com/@killeryzombie/video/7267349680894266630', 'https://www.tiktok.com/@comuna5.lla/video/7285557210980158725', 'https://www.tiktok.com/@elpelucamilei/video/7317458584999120133']


In [ ]:
# Ejemplo de uso
enlaces = extraccion_link('#milei', 10)
print(enlaces)

In [54]:
main(enlaces)

Nombre de usuario: @user9098712142591
Fecha del comentario: Hace 48 min
Cantidad de likes: 30
Comentario: Gracias Sra Bullrich,yo estoy absoluta segura que es así en el año 2016 le dejaron de entregar el medicamento oncológico a mi hermana,por qué usaron los recursos para la campaña del Sr. Sioli entonces
------
Nombre de usuario: @isabelrobledo17
Fecha del comentario: Hace 40 min
Cantidad de likes: 30
Comentario: siempre fue así el que cobra plan tiene que asistir a las marchas y si no chau...
------
Nombre de usuario: @andymoraleess
Fecha del comentario: Hace 1 h
Cantidad de likes: 50
Comentario: Gracias @Patricia Bullrich por el.empuje y calma que estas transmitiendo hoy 💙💙👏👏👏
------
Nombre de usuario: @laabuelabellaca52
Fecha del comentario: Hace 53 min
Cantidad de likes: 19
Comentario: Mi capacidad de Asombro no se agota
------
Nombre de usuario: @juancarlosvivasra
Fecha del comentario: Hace 45 min
Cantidad de likes: 1
Comentario: en Venezuela pasa igual 😒
------
Nombre de usuario

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.css-1i7ohvi-DivCommentItemContainer"}
  (Session info: chrome=121.0.6167.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF67DD75E42+3538674]
	(No symbol) [0x00007FF67D994C02]
	(No symbol) [0x00007FF67D845AEB]
	(No symbol) [0x00007FF67D88BF4E]
	(No symbol) [0x00007FF67D88C0CC]
	(No symbol) [0x00007FF67D8CE477]
	(No symbol) [0x00007FF67D8AF0EF]
	(No symbol) [0x00007FF67D8CBDE3]
	(No symbol) [0x00007FF67D8AEE53]
	(No symbol) [0x00007FF67D87F514]
	(No symbol) [0x00007FF67D880631]
	GetHandleVerifier [0x00007FF67DDA6CAD+3738973]
	GetHandleVerifier [0x00007FF67DDFC506+4089270]
	GetHandleVerifier [0x00007FF67DDF4823+4057299]
	GetHandleVerifier [0x00007FF67DAC5C49+720121]
	(No symbol) [0x00007FF67D9A126F]
	(No symbol) [0x00007FF67D99C304]
	

In [ ]:
#CSV to json
import pandas as pd

df = pd.read_csv("comentarios_tiktok.csv")
df.to_json("comentarios_tiktok.json")
